- Reference
  - https://blog.shikoan.com/resnet-multiple-framework/
  - https://github.com/koshian2/ResNet-MultipleFramework/blob/master/resnet_keras.py

In [1]:
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization, Add, Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.regularizers import l2

In [5]:
import time
import pickle
import os

In [6]:
# 経過時間用のコールバック
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_start_time)

In [13]:
class ResNet:
    def __init__(self, n, framework, channels_first=False, initial_lr=0.01, nb_epochs=100):
        self.n = n
        self.framework = framework

        # 論文通りの初期学習率=0.1だと発散するので0.01にする
        self.initial_lr = initial_lr
        self.nb_epochs = nb_epochs
        self.weight_decay = 0.0005

        # MX-Netではchannels_firstなのでその対応をする
        self.channels_first = channels_first
        self.data_format = "channels_first" if channels_first else "channels_last"
        self.bn_axis = 1 if channels_first else -1

        # Make model
        self.model = self.make_model()

    # オリジナルの論文に従って、サブサンプリングにPoolingではなくstride=2のConvを使う
    def subsumpling(self, output_channels, input_tensor):
        return Conv2D(output_channels, kernel_size=1, strides=(2,2), data_format=self.data_format, kernel_regularizer=l2(self.weight_decay))(input_tensor)

    # BN->ReLU->Conv->BN->ReLU->Conv をショートカットさせる(Kaimingらの研究による)
    # https://www.slideshare.net/KotaNagasato/resnet-82940994
    def block(self, channles, input_tensor):
        # ショートカット元
        shortcut = input_tensor

        # メイン側
        x = BatchNormalization(axis=self.bn_axis)(input_tensor)
        x = Activation("relu")(x)
        x = Conv2D(channles, kernel_size=3, padding="same", data_format=self.data_format, kernel_regularizer=l2(self.weight_decay))(x)
        x = BatchNormalization(axis=self.bn_axis)(x)
        x = Activation("relu")(x)
        x = Conv2D(channles, kernel_size=3, padding="same", data_format=self.data_format, kernel_regularizer=l2(self.weight_decay))(x)

        # 結合
        return Add()([x, shortcut])

    def make_model(self):
        input = Input(shape=(3, 32, 32)) if self.channels_first else Input(shape=(32, 32, 3))

        # 3->16にチャンネル数を増やす
        x = Conv2D(16, kernel_size=3, padding="same", data_format=self.data_format, kernel_regularizer=l2(self.weight_decay))(input)

        # 32x32x16のブロックをn回
        for i in range(self.n):
            x = self.block(16, x)

        # 16x16x32
        x = self.subsumpling(32, x)

        for i in range(self.n):
            x = self.block(32, x)

        # 8x8x64
        x = self.subsumpling(64, x)

        for i in range(self.n):
            x = self.block(64, x)

        # Global Average Pooling
        x = GlobalAveragePooling2D(data_format=self.data_format)(x)
        x = Dense(10, activation="softmax")(x)

        # model
        model = Model(input, x)

        return model

    def lr_schduler(self, epoch):
        x = self.initial_lr
        if epoch >= self.nb_epochs * 0.5: x /= 10.0
        if epoch >= self.nb_epochs * 0.75: x /= 10.0
        return x

    def train(self, X_train, y_train, X_val, y_val):
        # コンパイル
        self.model.compile(optimizer=SGD(lr=self.initial_lr, momentum=0.9), loss="categorical_crossentropy", metrics=["acc"])

        # Data Augmentation
        traingen = ImageDataGenerator(
            rescale=1./255,
            width_shift_range=4./32,
            height_shift_range=4./32,
            horizontal_flip=True)

        valgen = ImageDataGenerator(
            rescale=1./255)

        # Callback
        time_cb = TimeHistory()
        lr_cb = LearningRateScheduler(self.lr_schduler)

        # Train
        history = self.model.fit_generator(traingen.flow(X_train, y_train, batch_size=128), epochs=self.nb_epochs,
                                           steps_per_epoch=len(X_train)/128, validation_data=valgen.flow(X_val, y_val),
                                           callbacks=[time_cb, lr_cb]).history
        history["time"] = time_cb.times

        # Save history
        file_name = f"{self.framework}_n{self.n}.dat"
        file_path = os.path.join('data', file_name)
        #with open(file_name, "wb") as fp:
        with open(file_path, "wb") as fp:
            pickle.dump(history, fp)

In [16]:
# Main function
def main(n, framework):
    # layers = 6n+2
    net = ResNet(n, framework, nb_epochs=1)

    # CIFAR
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    print(X_train.shape, X_test.shape)
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)

    # train
    net.train(X_train, y_train, X_test, y_test)

In [17]:
main(3, "keras_tf")

(50000, 32, 32, 3) (10000, 32, 32, 3)
391/390 [==============================] - 283s 725ms/step - loss: 1.9770 - acc: 0.4053 - val_loss: 2.3470 - val_acc: 0.3373 - lr: 0.0100
